# 逻辑回归

In [1]:
import os
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import scipy.optimize as opt
%matplotlib inline

In [2]:
def getData(path, names=None):
    return pd.read_csv(path, header=None, names=names)
    pass

In [3]:
data = getData(os.getcwd() + "/../data/ex2data1.txt",["A", "B", "Y"])

In [4]:
data.head()

,A,B,Y
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1


In [5]:
data.describe()

,A,B,Y
count,100.000000,100.000000,100.000000
mean,65.644274,66.221998,0.600000
std,19.458222,18.582783,0.492366
min,30.058822,30.603263,0.000000
25%,50.919511,48.179205,0.000000
50%,67.032988,67.682381,1.000000
75%,80.212529,79.360605,1.000000
max,99.827858,98.869436,1.000000


In [6]:
data.insert(0,'ones',1) # 加入bias

In [7]:

'''归一化'''
data['A'] = (data['A']-data['A'].mean()) / data['A'].std()
data['B'] = (data['B']-data['B'].mean()) / data['B'].std()
data.head()

,ones,A,B,Y
0,1,-1.594216,0.635141,0
1,1,-1.817101,-1.201489,0
2,1,-1.531325,0.359483,0
3,1,-0.280687,1.080923,1
4,1,0.688062,0.490905,1


开始实现逻辑回归， 因为在推导公式中，为了方便计算，类别是以，1,-1分类的， 因此需要将0换为-1.
损失函数:$E_{in}(W)=\sum\limits_{n=1}^{N}ln(1+exp(-y_nW^Tx_n))$

In [8]:
def filterData(data):
    '''构造输入和标签'''
    col = data.shape[1]
    x = data.iloc[:,0:col-1]
    y = data.iloc[:,col-1:col].replace(0, -1) 
    return np.array(x.values), np.array(y.values)

def getWLin(X, Y):
    '''线性回归计算初始值'''
    return np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(Y)

def sigmod(s):
    '''sigmod函数'''
    return 1.0 / (1.0 + np.exp(-s))

def update(W, grad, ita):
    '''更新W'''
    return W - (ita * grad)

def MBGD(W, X, Y):
    '''批量梯度下降'''
    N = len(X)
    grad = np.zeros(X[0].shape)
    for i in range(N):
        sig = sigmod(-Y[i][0] * X[i].dot(W))
        grad += -Y[i][0] * sig * X[i]
    
    return grad / N

def SGD(W, X, Y, n):
    '''随机梯度'''
    sig = sigmod(-Y[n][0] * X[n].dot(W))
    return -Y[n][0] * sig * X[n]
    
    
def cost(W, X, Y):
    '''损失函数'''
    N = len(X)
    cost = 0.0
    for i in range(N):
        err = np.exp(-Y[i][0] * X[i].dot(W))
        cost += np.log(1 + err)
    
    return cost / N

def train_with_grad(X, Y, W, GD=1, iterations=500):
    '''训练'''
    N = len(X)
    for i in range(iterations):
        if GD:
            grad = MBGD(W, X, Y) # 全梯度
        else:
            grad = SGD(W, X, Y, i%N) # 随机梯度
        err = cost(W, X, Y)
        W = update(W, grad, 0.1)
    return W, err
    
def train_with_fmin(W):
    '''使用scipy的fmin优化'''
    W = np.matrix(np.zeros(X[0].shape))
    result = opt.fmin_tnc(func=cost, x0=W,fprime=MBGD, args=(X,Y))
    return result
    

In [9]:
'''使用随机梯度下降迭代求解'''
X, Y = filterData(data)
W = getWLin(X,Y).reshape(X[0].shape)
W_SGD = train_with_grad(X, Y, W, GD=0)

In [10]:
'''使用全梯度下降迭代求解'''
X, Y = filterData(data)
W = getWLin(X,Y).reshape(X[0].shape)
W_MBGD = train_with_grad(X, Y, W, GD=1)

In [11]:
'''使用第三方库scipy的fmin直接优化'''
X, Y = filterData(data)
W = getWLin(X,Y).reshape(X[0].shape)
W_fmin = train_with_fmin(W)

In [12]:
W_SGD, W_MBGD, W_fmin

((array([ 1.12902306,  2.51740545,  2.37171023]), 0.22272664575554293),
 (array([ 1.02284455,  2.5412209 ,  2.32502707]), 0.22322532581110394),
 (array([ 1.71881679,  4.01402376,  3.74529924]), 20, 1))

In [13]:
cost(W_fmin[0], X, Y) # 计算fmin的误差

0.20349771521035276

可以看出随机梯度和全梯度与直接使用fmin误差差不多，但是训练时间的话，fmin 最快，随机梯度次之，最慢的全梯度。因此以后的优化问题均使用fmin。

另外基于原数据作了一个简单的测试，正确率85%+

In [14]:
def predict(W, X, Y):
    '''预测'''
    predictions = [1 if sigmod(x.dot(W)) >= 0.5 else -1 for x in X]
    correct = [1 if ((a == 1 and b == 1) or (a == -1 and b == -1)) else 0 for (a, b) in zip(predictions, Y)]
    accuracy = (sum(map(int, correct)) % len(correct))
    return predictions, accuracy

In [15]:
result_fmin = predict(W_fmin[0], X, Y)
print 'fmin accuracy = {0}%'.format(result_fmin[1])

result_sgd = predict(W_SGD[0], X, Y)
print 'SGD accuracy = {0}%'.format(result_sgd[1])

result_mbgd = predict(W_MBGD[0], X, Y)
print 'MBGD accuracy = {0}%'.format(result_mbgd[1])

fmin accuracy = 89%
SGD accuracy = 89%
MBGD accuracy = 89%


    可以看出，正确率不到90%,因为我们没有加任何trick，比如正则化等等。下面我们再使用sklearn自带的lr模型试试如下:

In [17]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X, Y)
lr_result = LR.predict(X)
correct = [1 if ((a == 1 and b == 1) or (a == -1 and b == -1)) else 0 for (a, b) in zip(lr_result, Y)]
accuracy = (sum(map(int, correct)) % len(correct))
print 'Sklearn accuracy = {0}%'.format(accuracy)

Sklearn accuracy = 89%


大致差不多。